In [43]:
from pandas import read_csv, read_parquet, read_orc, DataFrame, concat
import numpy as np

In [44]:
#to warmup the gpu
df = DataFrame({'a':range(10)})

In [45]:
df.head()

,a
0,0
1,1
2,2
3,3
4,4


---
#### Reading the cal_dim table from `/data/calendar.csv` stored in csv format

In [46]:
%%time
cal = read_csv('/data/calendar.csv')
cal['cal_dt'] = cal['cal_dt'].astype('datetime64[ms]')
cal['ly_date'] = cal['ly_date'].astype('datetime64[ms]')
cal['ly_comp_visit_dt'] = cal['ly_comp_visit_dt'].astype('datetime64[ms]')

CPU times: user 7.18 ms, sys: 7.74 ms, total: 14.9 ms
Wall time: 11.2 ms


In [47]:
cal.shape

(1, 4)

In [48]:
cal.head()

,cal_dt,ly_date,ly_comp_visit_dt,wm_yr_wk_nbr
0,2017-04-11,2016-04-11,2016-04-12,201711


In [49]:
join_dates = concat([
        DataFrame({'cal_dt':cal['cal_dt'], 'cal_ind':col_ind , 'join_date':cal[col]}) 
        for (col_ind, col) in [('C', 'cal_dt'), ('L', 'ly_date'), ('R', 'ly_comp_visit_dt')]
    ])
join_dates.head()

,cal_dt,cal_ind,join_date
0,2017-04-11,C,2017-04-11
0,2017-04-11,L,2016-04-11
0,2017-04-11,R,2016-04-12


---
#### Reading the store_visit_scan table from `/data/store_visit_scan_mask.parquet` stored in parquet format

In [50]:
%%time
svs = read_parquet('/data/store_visit_scan_mask.parquet')
svs['visit_dt'] = svs['visit_dt'].astype('datetime64[ms]')

CPU times: user 2.83 s, sys: 467 ms, total: 3.3 s
Wall time: 2.27 s


In [21]:
print("shape : ", svs.shape)
svs.head()

shape :  (6599219, 8)


,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd
0,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US
1,710104362,1999,7,867174169,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US
2,710101771,17377,6,738523462,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US
3,710103327,9700,10,892230403,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US
4,710101361,8737,4,851258830,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US


---
#### Reading the store_visit table from `/data/store_visit_mask.parquet` stored in parquet format

In [22]:
%%time
vst = read_parquet('/data/store_visit_mask.parquet')
vst['visit_dt'] = vst['visit_dt'].astype('datetime64[ms]')

CPU times: user 2.54 s, sys: 384 ms, total: 2.93 s
Wall time: 2.17 s


In [23]:
print("shape : ", vst.shape)
vst.head()

shape :  (7189981, 5)


,store_nbr,visit_nbr,visit_tm,visit_dt,op_cmpny_cd
0,11680,710100004,1,2017-04-11,WMT-US
1,16237,710100004,1,2017-04-11,WMT-US
2,17365,710100004,2,2017-04-11,WMT-US
3,5071,710100005,3,2017-04-11,WMT-US
4,5443,710100005,3,2017-04-11,WMT-US


---
### Performing inner join between store_visit_scan and store_visit

In [24]:
%%time
# merging the scan and visite tables
df = svs.merge(vst, on=['visit_dt', 'store_nbr', 'visit_nbr'], how='inner')
df = df.drop_duplicates()

CPU times: user 4.02 s, sys: 721 ms, total: 4.74 s
Wall time: 4.74 s


In [25]:
print("shape : ", df.shape)
df.head()

shape :  (2338031, 10)


,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd_x,visit_tm,op_cmpny_cd_y
0,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US
1,710103784,1099,4,986506717,0,2017-06-23T12:46:36.000Z,2017-04-11,WMT-US,1546,WMT-US
2,710103507,4027,9,278938867,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1540,WMT-US
3,710103507,4027,10,278938867,0,2017-06-23T12:46:36.000Z,2017-04-11,WMT-US,1540,WMT-US
4,710103507,4027,11,770584414,0,2017-06-23T12:46:36.000Z,2017-04-11,WMT-US,1540,WMT-US


---
#### Reading the item_dim table from `/data/item_dim_mask.orc` stored in orc format

In [34]:
%%time
item = read_orc('/data/item_dim_mask.orc')

CPU times: user 2.16 s, sys: 612 ms, total: 2.77 s
Wall time: 2.44 s


In [35]:
print("shape : ", item.shape)
item.head()

shape :  (34110184, 4)


,mds_fam_id,item_nbr,upc_nbr,dept_nbr
0,-905,-6989,-1994.0,8
1,-1787,-6989,-1994.0,400
2,-1778,-6989,-1994.0,396
3,-1769,-6989,-1994.0,392
4,-1760,-6989,-1994.0,388


---
### Performing inner join with item_dim

In [36]:
%%time
df = df.rename(columns = {'scan_id':'mds_fam_id'})
df = df.merge(item, on=['mds_fam_id'], how='inner')

CPU times: user 11.2 s, sys: 2.6 s, total: 13.8 s
Wall time: 13.8 s


In [37]:
print("shape : ", df.shape)
df.head()

shape :  (6957414, 18)


,visit_nbr,store_nbr,scan_seq_nbr,mds_fam_id,scan_type,load_ts,cal_dt,op_cmpny_cd_x,visit_tm,op_cmpny_cd_y,item_nbr_x,upc_nbr_x,dept_nbr_x,cal_ind,join_date,item_nbr_y,upc_nbr_y,dept_nbr_y
0,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,C,2017-04-11,3882699650,1.020180e+10,12
1,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,L,2016-04-11,3882699650,1.020180e+10,12
2,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,R,2016-04-12,3882699650,1.020180e+10,12
3,710100640,1297,7,877534402,0,2017-06-23T12:47:44.000Z,2017-04-11,WMT-US,1007,WMT-US,3882699650,1.020180e+10,12,C,2017-04-11,3882699650,1.020180e+10,12
4,710100640,1297,7,877534402,0,2017-06-23T12:47:44.000Z,2017-04-11,WMT-US,1007,WMT-US,3882699650,1.020180e+10,12,L,2016-04-11,3882699650,1.020180e+10,12


In [38]:
%%time
df = df.rename(columns = {'visit_dt':'cal_dt'})
df = df.merge(join_dates, on=['cal_dt'], how='inner')

CPU times: user 4.21 s, sys: 1.65 s, total: 5.87 s
Wall time: 5.88 s


In [39]:
df.head()

,visit_nbr,store_nbr,scan_seq_nbr,mds_fam_id,scan_type,load_ts,cal_dt,op_cmpny_cd_x,visit_tm,op_cmpny_cd_y,item_nbr_x,upc_nbr_x,dept_nbr_x,cal_ind_x,join_date_x,item_nbr_y,upc_nbr_y,dept_nbr_y,cal_ind_y,join_date_y
0,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,C,2017-04-11,3882699650,1.020180e+10,12,C,2017-04-11
1,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,C,2017-04-11,3882699650,1.020180e+10,12,L,2016-04-11
2,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,C,2017-04-11,3882699650,1.020180e+10,12,R,2016-04-12
3,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,L,2016-04-11,3882699650,1.020180e+10,12,C,2017-04-11
4,710103784,1099,8,877534402,0,2017-06-23T12:46:35.000Z,2017-04-11,WMT-US,1546,WMT-US,3882699650,1.020180e+10,12,L,2016-04-11,3882699650,1.020180e+10,12,L,2016-04-11


### Perfroming groupby on store number to get the item counts

In [40]:
%%time
df_grpd = df.groupby('store_nbr').agg({'mds_fam_id':'count'})

CPU times: user 331 ms, sys: 110 ms, total: 440 ms
Wall time: 438 ms


In [41]:
df_grpd

,mds_fam_id
store_nbr,
7,6921
10,8478
13,6300
16,6579
19,4761
...,...
22051,1818
22075,1620
22087,1350


In [42]:
df_grpd = df_grpd.rename(columns = {'mds_fam_id':'mds_fam_id_count'})
df_grpd

,mds_fam_id_count
store_nbr,
7,6921
10,8478
13,6300
16,6579
19,4761
...,...
22051,1818
22075,1620
22087,1350


---
### Top 5 store with maximum item sold on given day

In [ ]:
df_grpd.mds_fam_id_count.nlargest(5).to_frame()

---
#### Total execution time:  
 = 4.79ms + 1.53s + 3.85s + 4.73s + 2.45s + 15.4s + 1.18s + 183ms  
 ~ 30s approx.